In [ ]:
import flapjack
import numpy as np

In [ ]:
fj = flapjack.Flapjack(url_base='localhost:8000')
fj.log_in(username='tim', password='chicken')

In [ ]:
def receiver(fp, A, alpha_A, K_A, mu, gamma, Dt):
    # Update protein concs
    a = (A/K_A)**2
    nextfp = fp + (alpha_A*a/(1 + a) - gamma*fp - mu*fp) * Dt
    return nextfp

def inverter(p_i, fp, alpha_j, K_i, n_i, gamma, mu, Dt):
    p = (p_i/K_i)**n_i
    nextfp = fp + ( alpha_j / ( 1 + p ) - gamma*fp - mu*fp ) * Dt
    return nextfp

In [ ]:
def step_receiver(K_A, alpha_A, gamma): 
    def func(p, signal, growth_rate, dt):
        # p[0] = fluo protein
        A = signal
        mu = growth_rate
        return [receiver(p[0], A, alpha_A, K_A, mu, gamma, dt)]
    return func

In [ ]:
def step_inverter(K_A, alpha_A, alpha_j, K_i, n_i, gamma):
    def func(p, signal, growth_rate, dt):
        # p[0] = fluo protein
        # p[1] = repressor

        mu = growth_rate
        A = signal
        
        # Receiver makes unstable repressor
        next_p1 = receiver(p[1], A, alpha_A, K_A, mu, gamma, dt)

        # Inverter makes stable FP
        next_p0 = inverter(p[1], p[0], alpha_j, K_i, n_i, mu, 0, dt)

        return [next_p0, next_p1]
    return func

In [ ]:
def step_double_inverter(K_A, 
                         alpha_A, 
                         alpha_j, 
                         K_i, 
                         n_i,
                         alpha_k,
                         K_j,
                         n_j,
                         gamma): 
    def func(p, signal, growth_rate, dt):
        # p[0] = fluo protein
        # p[1] = repressor i
        # p[2] = repressor j

        mu = growth_rate
        A = signal

        # Receiver makes unstable repressor i
        next_p1 = receiver(p[1], A, alpha_A, K_A, mu, gamma, dt)

        # Inverter 1 makes unstable repressor j
        next_p2 = inverter(p[1], p[2], alpha_j, K_i, n_i, mu, gamma, dt)

        # Inverter 2 makes stable FP
        next_p0 = inverter(p[2], p[0], alpha_k, K_j, n_j, mu, 0, dt)

        return [next_p0, next_p1, next_p2]
    return func

In [ ]:
concs = np.logspace(-6, 6, 24, endpoint=True)

In [ ]:
fj.get('chemical')
chemical = fj.get('chemical', name='A')
#chemical = fj.create('chemical', name='A', description='Simulated inducer')
chemical

### Simulate signal receiver

In [ ]:
sim = flapjack.Simulator(
    study_name='Receiver simulation alpha scan deg',
    assay_name='Receiver 1',
    study_description='Simulation of signal receiver',
    assay_description='Replicate 1',
    dna_name='pREC',
    init_proteins=[0],
    concentrations=concs
)
sim.create_meta_objects(fj)
K_A = 1
alpha_A = 1e2
sim.create_data(fj, step_receiver(K_A, alpha_A, 0), 4, 100, 24/100, 10)

### Simulate inverter with receiver

In [ ]:
K_i = 1
n_i = 2
i = 1
for gamma in np.linspace(0.5, 2, 3, endpoint=True):
    for alpha_j in np.logspace(2, 5, 4, endpoint=True):
        sim = flapjack.Simulator(
            study_name='TU characterization alpha/gamma scan 2',
            assay_name=f'Inverter, alpha_j={alpha_j}, K_i={K_i}, n_i={n_i}, gamma={gamma}',
            study_description='Simulation of repressilator TUs',
            assay_description='Replicate 1',
            dna_name=f'pINV{i}',
            init_proteins=[0,0],
            concentrations=concs
        )
        sim.create_meta_objects(fj)
        sim.create_data(
            fj, 
            step_inverter(K_A, alpha_A, alpha_j, K_i, n_i, gamma), 
            4, 100, 24/100, 10
        )
        i += 1
        

### Simulate double inverter with receiver

In [ ]:
alpha_k = 1e2
K_j = 1
n_j = 2
gamma = 1

for alpha_j in np.logspace(2, 5, 4, endpoint=True):
    sim = flapjack.Simulator(
        study_name='TU characterization alpha scan deg',
        assay_name=f'Double inverter, alpha_j={alpha_j}, K_j={K_j}, n_j={n_j}',
        study_description='Simulation of repressilator TUs',
        assay_description='Replicate 1',
        dna_name='pDINV',
        init_proteins=[0,0,0],
        concentrations=concs
    )
    sim.create_meta_objects(fj)
    step = step_double_inverter(
        K_A, alpha_A,
        alpha_j, K_i, n_i,
        alpha_k, K_j, n_j,
        gamma
    )
    sim.create_data(fj, step, 4, 200, 24/100, 10)